In [1]:
!wandb login 6c2c9e426d778cb5f8ebd597263a155a1c4bcb11

wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


In [2]:
import wandb
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re

api = wandb.Api()

entity = "cezary17"
project = "alsatian-quantized"
sweep_id_1 = "12eawo3s"

wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: cezary to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [3]:
sweep_1 = api.sweep(f"{entity}/{project}/{sweep_id_1}")

In [4]:
aggregated_data = []

def get_model_names(line: str):
    if line.startswith("Model snapshots found: "):
        model_names = line.split("Model snapshots found: ")[1]
        return eval(model_names)
    else:
        return None

for run in sweep_1.runs:
    training_dataset = run.config.get('dataset')
    model_dataset = run.config.get('model_dataset')

    file = run.file("output.log")
    file.download(root='resnet18-outputs', replace=True)

    with open('resnet18-outputs/output.log', 'r') as file:
        lines = file.readlines()
        model_names = get_model_names(lines[0])

        result_lines = []
        result_lines_quantized = []
        curr_quantized = False

        for line in lines:
            if line.startswith("Loss:"):
                if curr_quantized:
                    result_lines_quantized.append(line)
                else:
                    result_lines.append(line)
                curr_quantized = not curr_quantized
            elif line.startswith("Error loading model"):
                result_lines.append("ERROR")
                result_lines_quantized.append("ERROR")

        metrics_original = [
            (float(match.group(1)), float(match.group(2)))
            if (match := re.search(r"Loss: ([\d.]+), Acc: ([\d.]+)", line))
            else (np.nan, np.nan)
            for line in result_lines
        ]

        metrics_quantized = [
            (float(match.group(1)), float(match.group(2)))
            if (match := re.search(r"Loss: ([\d.]+), Acc: ([\d.]+)", line))
            else (np.nan, np.nan)
            for line in result_lines_quantized
        ]

        for model_name, (loss_orig, acc_orig), (loss_quant, acc_quant) in zip(model_names, metrics_original, metrics_quantized):
            aggregated_data.append({
                "training_dataset": training_dataset,
                "model_dataset": model_dataset,
                "model_name": model_name,
                "original_accuracy": acc_orig,
                "quantized_accuracy": acc_quant
            })

aggregated_df = pl.DataFrame(aggregated_data)

aggregated_df

training_dataset,model_dataset,model_name,original_accuracy,quantized_accuracy
str,str,str,f64,f64
"""imagenette""","""stanford-dogs""","""resnet18-ri-1-id-6i2V""",0.9361,0.9101
"""imagenette""","""stanford-dogs""","""resnet18-ri-1-id-P3fA""",0.9368,0.9121
"""imagenette""","""stanford-dogs""","""resnet18-ri-2-id-W0EJ""",0.8591,0.8354
"""imagenette""","""stanford-dogs""","""resnet18-ri-2-id-fjYZ""",0.8548,0.8535
"""imagenette""","""stanford-dogs""","""resnet18-ri-2-id-jihA""",0.8706,0.8446
…,…,…,…,…
"""image-woof""","""image-woof""","""resnet18-ri-2-id-gTpi""",0.9058,0.8022
"""image-woof""","""image-woof""","""resnet18-ri-3-id-3bLu""",0.8984,0.7949
"""image-woof""","""image-woof""","""resnet18-ri-3-id-8UYF""",0.8939,0.8203


In [8]:
aggregated_df.write_parquet("resnet18_sweep_data.parquet")
aggregated_df.write_csv("resnet18_sweep_data.csv")

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=7dd8a54b-b654-4e26-9ba8-1dc7a02da688' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>